# CNN-Based Satellite Image Feature Extraction

This notebook encodes satellite images into fixed length numerical representations
using a pretrained Convolutional Neural Network (CNN). These embeddings capture
high level visual neighborhood context and are later fused with tabular features
for multimodal property valuation.

## 1. Setup and Imports

We import PyTorch, TorchVision, and supporting libraries required for dataset
construction, image preprocessing, and CNN-based feature extraction.

In [1]:
import os
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from PIL import Image
from tqdm import tqdm

## 2. Loading Sampled Training Data

Satellite image embeddings are extracted only for the stratified subset of training
data created during preprocessing. This ensures computational efficiency while
maintaining representation across the price distribution.

In [2]:
train_df = pd.read_csv("../data/processed/train_sampled.csv")
IMG_DIR = "../data/images/train"

print(train_df.shape)

(5000, 21)


The dataset size confirms that embeddings will be generated for all sampled properties.

## 3. Image Preprocessing

Satellite images are resized and normalized using ImageNet statistics.
This preprocessing matches the requirements of the pretrained CNN and
ensures consistent feature extraction.

In [3]:
image_transforms = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225]
    )
])

## 4. Custom PyTorch Dataset

A custom Dataset class is defined to load satellite images and their corresponding
property IDs. The dataset returns images only (no labels), as the CNN is used purely
as a feature extractor.

In [4]:
class SatelliteDataset(Dataset):
    def __init__(self, df, img_dir, transform=None):
        self.df = df.reset_index(drop=True)
        self.img_dir = img_dir
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img_path = os.path.join(self.img_dir, f"{row['id']}.png")

        image = Image.open(img_path).convert("RGB")

        if self.transform:
            image = self.transform(image)

        return image, row["id"]

## 5. DataLoader Configuration

Images are loaded in batches without shuffling to preserve the mapping between
image embeddings and property IDs. Single threaded loading is used to ensure
compatibility with macOS environments.

In [5]:
BATCH_SIZE = 32

dataset = SatelliteDataset(
    train_df,
    IMG_DIR,
    transform=image_transforms
)

loader = DataLoader(
    dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,    # DO NOT SHUFFLE
    num_workers=0     # REQUIRED on macOS
)

## 6. Pretrained CNN Feature Extractor

We use a pretrained ResNet-18 model as a fixed feature extractor.
The final classification layer is removed, and all parameters are frozen to
prevent training and overfitting.

In [6]:
resnet = models.resnet18(pretrained=True)
resnet.fc = nn.Identity()  # remove classifier

for param in resnet.parameters():
    param.requires_grad = False

/opt/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


## 7. Device Configuration

The model is moved to GPU if available; otherwise, computation is performed on CPU.
The model is set to evaluation mode to disable dropout and batch normalization updates.

In [7]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

resnet = resnet.to(device)
resnet.eval()

Using device: cpu


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

## 8. Extracting Image Embeddings

Satellite images are passed through the CNN in batches to obtain 512-dimensional
feature vectors. Gradients are disabled to improve efficiency and ensure
deterministic inference.

In [8]:
all_embeddings = []
all_ids = []

with torch.no_grad():
    for images, ids in tqdm(loader):
        images = images.to(device)

        features = resnet(images)     # (B, 512)
        features = features.cpu().numpy()

        all_embeddings.append(features)
        all_ids.extend(ids.numpy())

100%|█████████████████████████████████████████| 157/157 [01:42<00:00,  1.54it/s]


Each satellite image is encoded into a fixed length embedding that captures
high level spatial and visual characteristics of the surrounding neighborhood.

## 9. Saving Encoded Features

The extracted image embeddings and corresponding property IDs are saved to disk
for reuse in downstream multimodal modeling without reprocessing images.

In [9]:
image_embeddings = np.vstack(all_embeddings)
image_ids = np.array(all_ids)

np.save("../data/processed/image_embeddings.npy", image_embeddings)
np.save("../data/processed/image_ids.npy", image_ids)

print("Embeddings shape:", image_embeddings.shape)
print("IDs shape:", image_ids.shape)

Embeddings shape: (5000, 512)
IDs shape: (5000,)


The final embedding matrix confirms successful encoding of all sampled images.
Each property is represented by a 512-dimensional visual feature vector.

## Summary

This notebook converts raw satellite images into compact numerical embeddings
using a pretrained CNN. These embeddings encode neighborhood level visual context
and serve as the image modality input for the multimodal property valuation model.